In [1]:
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from PIL import Image
from pytorch_lightning import seed_everything
from sklearn.cluster import DBSCAN
import cv2
from torchvision import transforms, utils
from torch.utils.data import DataLoader
import imagehash

from utils import hammingDist
from metric import getMetric

In [2]:
seed = 1
seed_everything(seed)

Global seed set to 1


1

In [3]:
train_df = pd.read_csv('dataset/train.csv')

In [4]:
tmp = train_df.groupby('label_group').posting_id.agg('unique').to_dict()
train_df['target'] = train_df.label_group.map(tmp)

In [5]:
train_df.head(3)

,posting_id,image,image_phash,title,label_group,target
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,"[train_129225211, train_2278313361]"
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,"[train_3386243561, train_3423213080]"
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,"[train_2288590299, train_3803689425]"


### Test phash

In [6]:
hammdist_df = pd.DataFrame(index=train_df.image_phash.values, columns=train_df.image_phash.values)

In [ ]:
count = hammdist_df.shape[0]
for i in hammdist_df.index:
    count += 1
    for c in hammdist_df.columns:
        hammdist_df.loc[i, c] = hammingDist(i, c)
    if count % 2000 == 0:
        hammdist_df.to_csv('hamming_dist.csv')
        print(count)
hammdist_df.to_csv('hamming_dist.csv')

### Get BERT emb of title

In [6]:
bert_embs = []

for i_batch, data in enumerate(post_dataloader):
#     print(i_batch, data['image'].shape, data['token_tensor'].shape, data['attn_mask'].shape)
    token_tensors = data['token_tensor'].cuda()
    attn_masks = data['attn_mask'].cuda()
    batch_embs = get_bert_embedding(token_tensors, attn_masks)
    bert_embs.append(batch_embs.cpu())

/home/chien/hdd/miniconda3/envs/shopee/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2068: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [17]:
embs = bert_embs[0]

for emb in bert_embs[1:]:
    embs = torch.cat((embs, emb), dim=0)
joblib.dump(embs, 'tmp/title_bert_embs.joblib')

In [26]:
title_embs = joblib.load('tmp/title_bert_embs.joblib')
title_embs.shape

torch.Size([34250, 768])

### Clustering

In [27]:
labels = DBSCAN(eps=0.05, min_samples=2, n_jobs=-1).fit_predict(title_embs)
uniq_labels = list(sorted(set(labels)))

# print(Counter(labels))

# for label in uniq_labels:
#     mask = (labels == label)
#     plt.scatter(reduced_2[mask, 0], reduced_2[mask, 1])
# plt.legend(uniq_labels, bbox_to_anchor=(1.05, 1))
# plt.tight_layout()
# plt.show()

NameError: name 'reduced_2' is not defined

In [28]:
len(uniq_labels)

1369

In [29]:
train_df['labels'] = labels

In [31]:
tmp = train_df.groupby('labels').posting_id.agg('unique').to_dict()
train_df['oof'] = train_df.labels.map(tmp)

In [32]:
def outlier_group(row):
    if row.labels == -1:
        return [row.posting_id]
    else:
        return row.oof
train_df['oof'] = train_df.apply(outlier_group, axis=1)

### Calculate metric

In [34]:
train_df['f1'] = train_df.apply(getMetric('oof'), axis=1)
print('CV score =', train_df.f1.mean())

CV score = 0.4832022085558331
